In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), '..','..','..'))
from configs import spark_config as config
from utils import spark_utils as utils

spark = utils.create_spark_session('merge_player_stats')

directory = config.PROCESSED_DATA_DIR

matches = spark.read.csv(os.path.join(directory, 'matches.csv'), header=True, inferSchema=True)
matchPlayers = spark.read.csv(os.path.join(directory, 'match_players.csv'), header=True, inferSchema=True).sort('match_id')
playerStats = spark.read.csv(os.path.join(directory, 'player_stats.csv'), header=True, inferSchema=True)
playerStats.show(5)

/home/ravikumar/miniconda3/envs/t20i/lib/python3.12/site-packages/airflow/configuration.py:859 FutureWarning: section/key [core/sql_alchemy_conn] has been deprecated, you should use[database/sql_alchemy_conn] instead. Please update your `conf.get*` call to use the new name

[2024-11-24T14:34:19.856+0530] {spark_utils.py:17} INFO - Creating Spark session.


your 131072x1 screen size is bogus. expect trouble
24/11/24 14:34:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


[2024-11-24T14:34:23.424+0530] {spark_utils.py:37} INFO - Spark session created successfully.


+---------------+----------------+-------+-------------+--------------+------+----------------+---------------+-----------------------+---------------+---------------+--------------+-------------+-------------+--------------+
|         Player|         Country| Season|Cum Mat Total|Cum Runs Total|Cum SR|Cumulative Overs|Cumulative Mdns|Cumulative Bowling Runs|Cumulative Wkts|Cumulative Econ|Cumulative Dis|Cumulative Ct|Cumulative St|Cumulative D/I|
+---------------+----------------+-------+-------------+--------------+------+----------------+---------------+-----------------------+---------------+---------------+--------------+-------------+-------------+--------------+
| AB de Villiers|    South Africa|   2017|           73|          1457|128.93|             0.0|            0.0|                    0.0|            0.0|            0.0|          68.0|         61.0|          7.0|          0.93|
|    TA Blundell|     New Zealand|2017/18|            1|             0|   0.0|             0.0| 

In [2]:
matchPlayers = matchPlayers.withColumnRenamed("country", "Country") \
                           .withColumnRenamed("player", "Player") \
                           .withColumnRenamed("season", "Season")

In [3]:
# Include row_num in the join
matchPlayersStats = matchPlayers.join(playerStats, on=['Player',"Country",'Season'], how='inner')
matchPlayersStats = matchPlayersStats.sort("match_id")

# Display the result starting from the 45th row
matchPlayersStats.show(44)

+--------------+-----------+-------+---------+--------+-------------+--------------+------+----------------+---------------+-----------------------+---------------+---------------+--------------+-------------+-------------+--------------+
|        Player|    Country| Season|player_id|match_id|Cum Mat Total|Cum Runs Total|Cum SR|Cumulative Overs|Cumulative Mdns|Cumulative Bowling Runs|Cumulative Wkts|Cumulative Econ|Cumulative Dis|Cumulative Ct|Cumulative St|Cumulative D/I|
+--------------+-----------+-------+---------+--------+-------------+--------------+------+----------------+---------------+-----------------------+---------------+---------------+--------------+-------------+-------------+--------------+
|     A Symonds|  Australia|   2005| bd77eb62|  211028|            1|            32|246.15|             3.0|            0.0|                   33.0|            1.0|           11.0|           0.0|          0.0|          0.0|           0.0|
|     MJ Clarke|  Australia|   2005| f842c2c

In [4]:
from pyspark.sql.functions import col
match_id = matchPlayersStats.groupBy('match_id').count().filter(col('count') == 22).select('match_id')
match_id_list = match_id.collect()
len(match_id_list)

1183

In [5]:
# Extract match_id values from the collected rows
match_id_values = [row.match_id for row in match_id_list]

# Filter matchPlayersStats using the extracted match_id values
matchPlayersStats = matchPlayersStats.filter(col('match_id').isin(match_id_values))
matchPlayersStats.show(5)

+------------+---------+------+---------+--------+-------------+--------------+------+----------------+---------------+-----------------------+---------------+---------------+--------------+-------------+-------------+--------------+
|      Player|  Country|Season|player_id|match_id|Cum Mat Total|Cum Runs Total|Cum SR|Cumulative Overs|Cumulative Mdns|Cumulative Bowling Runs|Cumulative Wkts|Cumulative Econ|Cumulative Dis|Cumulative Ct|Cumulative St|Cumulative D/I|
+------------+---------+------+---------+--------+-------------+--------------+------+----------------+---------------+-----------------------+---------------+---------------+--------------+-------------+-------------+--------------+
|AC Gilchrist|Australia|  2005| 2b6e6dec|  211028|            1|             1| 33.33|             0.0|            0.0|                    0.0|            0.0|            0.0|           0.0|          0.0|          0.0|           0.0|
|  RT Ponting|Australia|  2005| 7d415ea5|  211028|            1|

In [6]:
matchPlayersStats = matchPlayersStats.drop('Country','Player','Season','Player','Country',"player_id")
matchPlayersStats.show()

+--------+-------------+--------------+------+----------------+---------------+-----------------------+---------------+---------------+--------------+-------------+-------------+--------------+
|match_id|Cum Mat Total|Cum Runs Total|Cum SR|Cumulative Overs|Cumulative Mdns|Cumulative Bowling Runs|Cumulative Wkts|Cumulative Econ|Cumulative Dis|Cumulative Ct|Cumulative St|Cumulative D/I|
+--------+-------------+--------------+------+----------------+---------------+-----------------------+---------------+---------------+--------------+-------------+-------------+--------------+
|  211028|            1|             1| 33.33|             0.0|            0.0|                    0.0|            0.0|            0.0|           0.0|          0.0|          0.0|           0.0|
|  211028|            0|             0|   0.0|             0.0|            0.0|                    0.0|            0.0|            0.0|           0.0|          0.0|          0.0|           0.0|
|  211028|            0|      

In [7]:
num_rows = matchPlayersStats.count()
num_cols = len(matchPlayersStats.columns)
(num_rows, num_cols)

(26026, 13)

In [8]:
utils.spark_save_data(matchPlayersStats,config.MERGED_DATA_DIR, "player_stats.csv")
spark.stop()

[2024-11-24T14:35:18.786+0530] {spark_utils.py:64} INFO - Successfully wrote data to /usr/ravi/t20/data/3_mergedData/player_stats.csv
